In [23]:
%load_ext sql
# connect to db
%sql postgresql://postgres:postgres@db:5432/postgres

In [22]:
from public_data.models import ZoneUrba, Commune, ArtifAreaZoneUrba

z = ZoneUrba.objects.get(id=762778)
Commune.objects.intersect(z.mpoly)

ValueError: 'sql' was not found in history, as a file, url, nor in the user namespace.

In [8]:
from django.db.models import Prefetch, F
from project.models import Project

diagnostic = Project.objects.get(id=1504)
year = 2019
prefetcher = Prefetch("artifareazoneurba_set", queryset=ArtifAreaZoneUrba.objects.filter(year=year))
qs = ZoneUrba.objects.intersect(diagnostic.combined_emprise).prefetch_related(prefetcher)
qs.count(), qs

(1856,
 <QuerySet [<ZoneUrba: 32162 U 2.1841Ha>, <ZoneUrba: 32402 A 0.8140Ha>, <ZoneUrba: 32382 A 1.6528Ha>, <ZoneUrba: 32382 U 0.0150Ha>, <ZoneUrba: 32382 A 1.9692Ha>, <ZoneUrba: 32382 A 0.9210Ha>, <ZoneUrba: 32083 N 0.3255Ha>, <ZoneUrba: 32059 A 0.1929Ha>, <ZoneUrba: 32083 N 10.2648Ha>, <ZoneUrba: 32083 A 0.5538Ha>, <ZoneUrba: 32083 A 0.3306Ha>, <ZoneUrba: 32083 A 0.4833Ha>, <ZoneUrba: 32083 A 0.0300Ha>, <ZoneUrba: 32083 A 0.3704Ha>, <ZoneUrba: 32083 A 0.1751Ha>, <ZoneUrba: 32083 A 0.1493Ha>, <ZoneUrba: 32083 A 1.0497Ha>, <ZoneUrba: 32083 A 0.0630Ha>, <ZoneUrba: 32382 AUc 0.1137Ha>, <ZoneUrba: 32382 U 4.4532Ha>, '...(remaining elements truncated)...']>)

In [37]:
from django.db.models import Sum, Count

qs = (
    ArtifAreaZoneUrba.objects.filter(zone_urba__in=ZoneUrba.objects.intersect(diagnostic.combined_emprise))
    .filter(year__in=[2016, 2019])  # first and last ocsge
    .order_by("zone_urba__typezone", "year")
    .values("zone_urba__typezone", "year")
    .annotate(
        artif_area=Sum("area"),
        total_area=Sum("zone_urba__area"),
        nb_zones=Count("zone_urba_id"),
    )
)
qs

<QuerySet [{'zone_urba__typezone': 'A', 'year': 2016, 'artif_area': Decimal('877.8247'), 'total_area': Decimal('20385.5585'), 'nb_zones': 626}, {'zone_urba__typezone': 'A', 'year': 2019, 'artif_area': Decimal('882.2287'), 'total_area': Decimal('20385.7734'), 'nb_zones': 627}, {'zone_urba__typezone': 'AUc', 'year': 2016, 'artif_area': Decimal('33.9506'), 'total_area': Decimal('250.8629'), 'nb_zones': 63}, {'zone_urba__typezone': 'AUc', 'year': 2019, 'artif_area': Decimal('46.1665'), 'total_area': Decimal('256.1524'), 'nb_zones': 64}, {'zone_urba__typezone': 'AUs', 'year': 2016, 'artif_area': Decimal('17.3476'), 'total_area': Decimal('163.8537'), 'nb_zones': 27}, {'zone_urba__typezone': 'AUs', 'year': 2019, 'artif_area': Decimal('17.3234'), 'total_area': Decimal('163.8537'), 'nb_zones': 27}, {'zone_urba__typezone': 'N', 'year': 2016, 'artif_area': Decimal('492.8827'), 'total_area': Decimal('5488.9660'), 'nb_zones': 517}, {'zone_urba__typezone': 'N', 'year': 2019, 'artif_area': Decimal('4

In [16]:
ZoneUrba.objects.intersect(diagnostic.combined_emprise).first().id

1349682

In [19]:
zone = ZoneUrba.objects.get(id=1349682)
geom = zone.mpoly.transform("2154", clone=True)
geom.area

1183.2267009415218

In [32]:
zone.artifareazoneurba_set.all()

<QuerySet [<ArtifAreaZoneUrba: 1349682 2016 0.1108Ha>, <ArtifAreaZoneUrba: 1349682 2019 0.1108Ha>]>

In [35]:
query = (
    "insert into public_data_artifareazoneurba (zone_urba_id, year, area) "
    "select pdz.id, pdo.year, ST_Area(ST_Transform(ST_Union(ST_Intersection(pdo.mpoly, pdz.mpoly)), 2154)) / 10000 as artificial_area "
    "from public_data_zoneurba pdz left join public_data_artifareazoneurba pda on pda.zone_urba_id = pdz.id "
    "inner join public_data_ocsge pdo on ST_Intersects(pdo.mpoly, pdz.mpoly) and is_artificial = true "
    "where pda.id is null "
    "group by pdz.id, pdo.year;"
)
%sql $query

 * postgresql://postgres:***@db:5432/postgres
0 rows affected.


[]